###2

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
pd.options.display.float_format = '{:.2f}'.format

# Load datasets
akg_bayi = pd.read_csv('/content/AKG-bayi.csv')
food_check = pd.read_csv('/content/food-check.csv')

# Filter relevant columns
akg_columns = [col for col in akg_bayi.columns if 'Data.' in col]
food_columns = [col for col in food_check.columns if 'Data.' in col]
common_columns = list(set(akg_columns) & set(food_columns))

# Normalize data
scaler = MinMaxScaler()
akg_bayi_filtered = akg_bayi[['age'] + common_columns].copy()
akg_bayi_filtered[common_columns] = scaler.fit_transform(akg_bayi_filtered[common_columns])

food_check_filtered = food_check[['Category', 'Description'] + common_columns].copy()
food_check_filtered[common_columns] = scaler.transform(food_check[common_columns])

# Prepare training data for binary classification (nutrisi tercukupi atau tidak)
def generate_labels(akg_data, food_data):
    labels = []
    for _, row in akg_data.iterrows():
        age = row['age']
        akg_values = row[common_columns]
        total_nutrition = food_data[common_columns].sum()
        label = int((total_nutrition >= akg_values).all())  # 1 jika semua nutrisi tercukupi, 0 jika tidak
        labels.append(label)
    return np.array(labels)

labels = generate_labels(akg_bayi_filtered, food_check_filtered)

X = akg_bayi_filtered[common_columns].values
y = labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Keras model
model = Sequential([
    Input(shape=(len(common_columns),)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=50, batch_size=16, verbose=1)

# Save the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('/content/food_recommendation_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Define function to check if foods meet daily nutrition requirements
def check_food_nutrition():
    try:
        # User input
        age = int(input("Masukkan usia anak (dalam bulan): "))
        foods = input("Masukkan kategori makanan: ").strip()
        descriptions = input("Masukkan deskripsi makanan: ").strip()

        # Find AKG data for the specified age
        akg_row = akg_bayi_filtered[akg_bayi_filtered['age'] == age]
        if akg_row.empty:
            print("Tidak ada data AKG untuk usia yang dimasukkan.")
            return
        akg_values = akg_row[common_columns].iloc[0]

        # Filter food data for the selected foods
        selected_foods = food_check_filtered[
            (food_check_filtered['Category'].str.strip() == foods) &
            (food_check_filtered['Description'].str.strip() == descriptions)
        ]

        if selected_foods.empty:
            print("Tidak ada data makanan yang cocok dengan input.")
            return

        # Calculate total nutrition
        total_nutrition = selected_foods[common_columns].sum()

        # Calculate remaining nutrition required
        remaining_nutrition = akg_values - total_nutrition

        # Predict using the TFLite model
        interpreter = tf.lite.Interpreter(model_path='/content/food_recommendation_model.tflite')
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        total_nutrition_scaled = scaler.transform([total_nutrition.values])
        input_data = np.array(total_nutrition_scaled, dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        prediction = interpreter.get_tensor(output_details[0]['index'])[0][0]
        result = "terpenuhi" if prediction >= 0.5 else "tidak terpenuhi"

        # Display results
        # Format data nutrisi untuk usia tertentu
        print(f"Kebutuhan nutrisi untuk usia {age} bulan:")
        print(akg_values.round(2))  # 2 desimal

        # Total nutrisi yang dikonsumsi dari makanan
        print(f"Total nutrisi dari makanan yang dikonsumsi:")
        print(total_nutrition.round(2))  # 2 desimal

        # Selisih nutrisi yang masih dibutuhkan
        print(f"Selisih nutrisi yang masih dibutuhkan:")
        print(remaining_nutrition.round(2))  # 2 desimal

    except ValueError:
        print("Input tidak valid. Harap masukkan usia berupa angka.")
# Run the nutrition check
check_food_nutrition()


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.9637 - loss: 0.5104 - val_accuracy: 1.0000 - val_loss: 0.3252
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.3685 - val_accuracy: 1.0000 - val_loss: 0.2070
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.2705 - val_accuracy: 1.0000 - val_loss: 0.1257
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.2085 - val_accuracy: 1.0000 - val_loss: 0.0712
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.1787 - val_accuracy: 1.0000 - val_loss: 0.0394
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.1457 - val_accuracy: 1.0000 - val_loss: 0.0216
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.1119 - val_accuracy: 1.0000 - val_loss: 0.0119
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.1241 - val_accuracy: 1.0000 - val_loss: 0.0066

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
